In [1]:
from sqlalchemy import (create_engine, Column, ForeignKey,
                        Integer, MetaData, String, Table)

# Remove any existing database created in previous runs of this notebook.
import os

if os.path.isfile('tutorial.db'):
    os.unlink('tutorial.db')

### Set up a database "engine" that links to a database and populate it with data

In [2]:
engine = create_engine('sqlite:///tutorial.db')

metadata = MetaData(bind=engine)

users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String(40)),
              Column('age', Integer),
              Column('password', String))
 
addresses = Table('addresses', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('user_id', None, ForeignKey('users.id')),
                  Column('email_address', String, nullable=False))
 
# Create tables in database.
# You can call it as often as you like. It always checks for the existence 
# of the specified table before trying to create it.
metadata.create_all()

### Connecting to an existing table

In [3]:
# "autoload = True" loads the column name and types from the existing table.
users_again = Table("users", metadata, autoload=True)

### Inserting values

In [4]:
# `users.insert()` Create an Insert object on the table `users`
# The `values` method fills in the data to be used.
insert1 = users.insert().values(name="Joe", age=20, password="pass")

# The object does not communicate with the DB, it is a programmatic
# way of creating a SQL command.
# We can look at the command we have created by printing the object.
print insert1
# The placeholders will automatically be bound with the stored values.
print insert1.parameters

INSERT INTO users (name, age, password) VALUES (?, ?, ?)
{'age': 20, 'password': 'pass', 'name': 'Joe'}


In [5]:
# Create three insert commands...
insert1 = users.insert().values(name="Joe", age=20, password="pass")
insert2 = users.insert().values(name="Martha", age=45, password="dingbat")
insert3 = users.insert().values(name="Martin", age=33, password="1234")

# ... and commit them in the same transaction to the DB.
with engine.connect() as conn:
    conn.execute(insert1)
    conn.execute(insert2)
    conn.execute(insert3)

# We did not have to write any SQL!

In [7]:
# Insert multiple items into the `addresses` table.
with engine.connect() as conn:
    # Here we pass the values in a list.
    conn.execute(addresses.insert(), [ 
        {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
        {'user_id': 1, 'email_address' : 'jack@msn.com'},
        {'user_id': 2, 'email_address' : 'www@www.org'},
        {'user_id': 2, 'email_address' : 'wendy@aol.com'},
    ])

### SQL queries using the Core Expression Language

In [8]:
from sqlalchemy.sql import select

# Create a Select object to select all values from users
s = select([users])

# Again, we can look at the SQL command that the object represents
print s

SELECT users.id, users.name, users.age, users.password 
FROM users


In [9]:
# Execute the command.
with engine.connect() as conn:
    result = conn.execute(select([users]))

    print 'All Users (one by one):'
    for row in result:
        print row

All Users (one by one):
(1, u'Joe', 20, u'pass')
(2, u'Martha', 45, u'dingbat')
(3, u'Martin', 33, u'1234')


In [10]:
# Select just the names and the ages from the `users` table.
# `users.c` is the object representing all columns for table `users`
s = select([users.c.name, users.c.age])
print s

SELECT users.name, users.age 
FROM users


In [11]:
with engine.connect() as conn:
    result = conn.execute(select([users.c.name, users.c.age]))

    print 'User names and ages:'
    for row in result: 
        print row

User names and ages:
(u'Joe', 20)
(u'Martha', 45)
(u'Martin', 33)


In [12]:
# Combining tables with rows that have matching IDs.
s = select([users, addresses]).where(users.c.id==addresses.c.user_id)
print s

SELECT users.id, users.name, users.age, users.password, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id


In [13]:
with engine.connect() as conn:
    result = conn.execute(s)

    print 'Users and Addresses:'
    for row in result:
        print row

Users and Addresses:
(1, u'Joe', 20, u'pass', 1, 1, u'jack@yahoo.com')
(1, u'Joe', 20, u'pass', 2, 1, u'jack@msn.com')
(2, u'Martha', 45, u'dingbat', 3, 2, u'www@www.org')
(2, u'Martha', 45, u'dingbat', 4, 2, u'wendy@aol.com')


In [14]:
# Using logical operators in the selection.

from sqlalchemy.sql import and_, or_, not_

with engine.connect() as conn:
    s = select([users, addresses]).where(and_(users.c.id==addresses.c.user_id,
                                              users.c.age>25))
    result = conn.execute(s)

    print 'Users and Addresses:'
    for row in result:
        print row

Users and Addresses:
(2, u'Martha', 45, u'dingbat', 3, 2, u'www@www.org')
(2, u'Martha', 45, u'dingbat', 4, 2, u'wendy@aol.com')


In [15]:
# Count how many users are older than 21.
s = select([users.c.id]).where(users.c.age>21).count()
with engine.connect() as conn:
    print conn.execute(s).fetchone()

(2,)


In [16]:
# Updating an existing row
from sqlalchemy.sql import update

# Create an Update object, etc...
with engine.connect() as conn:
    u = update(users).where(users.c.id==1).values(age=21)
    conn.execute(u)

with engine.connect() as conn:
    s = select([users]).where(users.c.id==1)
    print conn.execute(s).fetchone()

(1, u'Joe', 21, u'pass')


### Using join

In [17]:
# Clearly, we need a Join object representing a JOIN SQL statement
user_address_join = users.join(addresses)
print user_address_join

users JOIN addresses ON users.id = addresses.user_id


In [19]:
with engine.connect() as conn:
    s = select([users.c.name, addresses.c.email_address], from_obj=user_address_join)
    result = conn.execute(s)

    print 'Users and Addresses:'
    for row in result:
        print row

Users and Addresses:
(u'Joe', u'jack@yahoo.com')
(u'Joe', u'jack@msn.com')
(u'Martha', u'www@www.org')
(u'Martha', u'wendy@aol.com')
